# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы "smart" и "ultra". Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 


In [15]:
import pandas as pd


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

## Откройте и изучите файл

In [16]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [17]:
df.sample(5)

,calls,minutes,messages,mb_used,is_ultra
2392,69.0,422.80,28.0,25065.10,0
1828,77.0,503.29,28.0,15346.17,0
2768,52.0,331.84,56.0,13217.21,0
1460,86.0,643.92,0.0,25872.85,0
119,40.0,258.20,20.0,13804.51,0


In [18]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Все данные числовые, среднее немного смещено вправо от медианы.


## Разбейте данные на выборки

Выделю в целевую выборку столбец значений 'is_ultra', поскольку цель - принимать решение о необходимом тарифе на основе параметров использования связи. Из выборки с исходными данными этот столбец исключаю.

In [19]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Мне нужно из всех предоставленных данных создать три типа выборок - для обучения, для тестирования и для контрольной проверки.
Рекомендуемые пропорции - 3 : 1 : 1, поэтому я сначала отделю от обучающей выборки тестовую и контрольную (60 и 40 %), а потом меньшую поделю пополам.

In [6]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.4, random_state=0) # отделяю 40% данных для валидационной и тестовой выборки

In [20]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=0) # отделяю 50% данных от валидационной для тестовой выборки

In [8]:
display(features_train, target_train, features_valid, target_valid, features_test, target_test)

,calls,minutes,messages,mb_used
386,19.0,147.72,20.0,19476.83
651,18.0,147.93,18.0,1904.01
2415,83.0,620.75,0.0,17914.78
805,40.0,232.35,37.0,13941.60
2488,77.0,566.52,20.0,13180.37
...,...,...,...,...
763,78.0,523.42,40.0,12477.65
835,70.0,519.74,28.0,24750.54
1653,80.0,538.39,51.0,21691.46
2607,54.0,396.41,11.0,21085.67


386     0
651     1
2415    0
805     0
2488    0
       ..
763     1
835     0
1653    0
2607    0
2732    0
Name: is_ultra, Length: 1928, dtype: int64

,calls,minutes,messages,mb_used
2409,36.0,277.79,0.0,11990.31
891,56.0,404.18,91.0,16476.40
128,44.0,359.39,43.0,15583.93
2390,3.0,48.74,2.0,439.62
1826,93.0,534.60,124.0,21252.19
...,...,...,...,...
64,67.0,390.62,99.0,12991.39
2580,55.0,337.04,31.0,17762.52
2935,88.0,642.31,170.0,44296.86
1217,38.0,244.50,27.0,22060.68


2409    0
891     0
128     0
2390    1
1826    0
       ..
64      0
2580    0
2935    1
1217    0
493     0
Name: is_ultra, Length: 643, dtype: int64

,calls,minutes,messages,mb_used
2329,86.0,580.55,33.0,8289.21
1791,21.0,112.98,115.0,24994.41
2481,77.0,551.43,56.0,7694.58
2537,97.0,788.48,0.0,23525.63
1768,70.0,497.55,66.0,24918.49
...,...,...,...,...
530,9.0,34.29,95.0,13403.98
688,111.0,633.05,94.0,8792.86
682,78.0,515.89,49.0,17368.65
6,57.0,431.64,20.0,3738.90


2329    0
1791    1
2481    0
2537    1
1768    0
       ..
530     1
688     1
682     0
6       1
913     1
Name: is_ultra, Length: 643, dtype: int64

Визуально разделение прошло хорошо.

## Исследуйте модели

Везде фиксирую случайную последовательность параметром random_state=0.

### Дерево решений

В цикле нахожу лучшую модель, меняя параметр глубины max_depth.

In [9]:
best_model = None
best_result = 10000
best_depth = 0
for depth in range(1, 6):
    model1 = DecisionTreeClassifier(random_state=0, max_depth=depth)
    model1.fit(features_train, target_train) # обучение на тренировочной выборке
    predictions_valid = model1.predict(features_valid) # предсказание на валидационной выборке
    result = mean_squared_error(target_valid, predictions_valid)**0.5 # значение метрики rmse на валидационной выборке
    if result < best_result:
        best_model = model1
        best_result = result
        best_depth = depth
model1 = best_model
print("RMSE наилучшей модели на валидационной выборке:", best_result, "Глубина дерева:", best_depth)



RMSE наилучшей модели на валидационной выборке: 0.44616910922652836 Глубина дерева: 3


### Случайный лес

В цикле нахожу лучшую модель, меняя параметр глубины max_depth и количества деревьев n_estimators.

In [21]:
best_model = None
best_result = 10000
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model2 = RandomForestClassifier(random_state=0, n_estimators=est, max_depth=depth)
        model2.fit(features_train, target_train) # обучение на тренировочной выборке
        predictions_valid = model2.predict(features_valid) # предсказание на валидационной выборке
        result = mean_squared_error(target_valid, predictions_valid)**0.5 # значение метрики rmse на валидационной выборке
        if result < best_result:
            best_model = model2
            best_result = result
            best_est = est
            best_depth = depth
model2 = best_model
print("RMSE наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)


RMSE наилучшей модели на валидационной выборке: 0.4229055977810245 Количество деревьев: 50 Максимальная глубина: 10


### Логистическая регрессия

In [11]:
model3 = LogisticRegression()
model3.fit(features_train, target_train) # обучение на тренировочной выборке
predictions_valid = model3.predict(features_valid) # предсказание на валидационной выборке

result = mean_squared_error(target_valid, predictions_valid)**0.5 # значение метрики RMSE на валидационной выборке

print("RMSE модели логистической регрессии на валидационной выборке:", result)


RMSE модели логистической регрессии на валидационной выборке: 0.5065665230763978


Меньшую ошибку даёт случайный лес с количеством деревьев 50 и максимальной глубиной 10.

## Проверьте модель на тестовой выборке

Посмотрим на другую метрику - accuracy. По заданию надо подобрать модель со значением этой метрики более 0,75.

### Дерево решений с глубиной 3.

In [22]:
predictions_test = model1.predict(features_test)

acc = accuracy_score(predictions_test, target_test)

print("Accuracy модели на тестовой выборке:", acc)

Accuracy модели на тестовой выборке: 0.7713841368584758


### Случайный лес к количеством деревьев 50 и глубиной 10

In [23]:
predictions_test = model2.predict(features_test)

acc = accuracy_score(predictions_test, target_test)

print("Accuracy модели на тестовой выборке:", acc)


Accuracy модели на тестовой выборке: 0.7822706065318819


### Логистическая регрессия

In [24]:
predictions_test = model3.predict(features_test)

acc = accuracy_score(predictions_test, target_test)

print("Accuracy модели на тестовой выборке:", acc)

Accuracy модели на тестовой выборке: 0.7542768273716952


Получается, что лучше всего подходит модель случайного леса на 50 деревьев глубиной 10.

## (бонус) Проверьте модели на адекватность

Я взяла для обучения 60% выборки. Адекватной будет считаться модель с accurcy выше 0,6. У всех трёх моделей условие выполнено.
Лучшая метрика у model2 - случайного леса с количеством деревьев 50 и глубиной 10.